In [2]:
# labeledDataToblockData
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import stft

In [3]:
# 生成含有触碰类型的数据块
import os
import pandas as pd

global_block_id = 0

def process_labeledDta(file_path, tactile_type):
    """处理单个labeled_data.csv文件，将连续label为1的数据块标记为指定触碰类型"""
    global global_block_id
    df = pd.read_csv(file_path)
    labeled_blocks = []

    in_block = False
    block_start = 0
    block_info = []

    for i in range(len(df)):
        if df.iloc[i, df.columns.get_loc('label')] == 1 and not in_block:
            in_block = True
            block_start = i
        elif df.iloc[i, df.columns.get_loc('label')] == 0 and in_block:
            in_block = False
            block = df.iloc[block_start:i].copy()
            block['touch_type'] = tactile_type
            block['block_id'] = global_block_id  # 添加全局唯一数据块标识符
            labeled_blocks.append(block)
            block_info.append((global_block_id, block_start, i - 1))  # 记录块的ID、开始行和结束行
            global_block_id += 1  # 增加全局唯一数据块标识符

    if in_block:
        block = df.iloc[block_start:].copy()
        block['touch_type'] = tactile_type
        block['block_id'] = global_block_id  # 添加全局唯一数据块标识符
        labeled_blocks.append(block)
        block_info.append((global_block_id, block_start, len(df) - 1))  # 记录块的ID、开始行和结束行
        global_block_id += 1  # 增加全局唯一数据块标识符
    
    return labeled_blocks, block_info

def calculate_zero_counts(df, block_info):
    """计算相邻数据块之间0的个数"""
    zero_counts = []
    for i in range(len(block_info) - 1):
        end_of_current_block = block_info[i][2]
        start_of_next_block = block_info[i + 1][1]
        zero_count = (df.iloc[end_of_current_block + 1:start_of_next_block]['label'] == 0).sum()
        zero_counts.append((block_info[i][0], block_info[i + 1][0], zero_count))
    return zero_counts

def determine_double_taps(zero_counts):
    """根据0的个数决定哪些单拍应该合并为双拍"""
    zero_counts_values = [count[2] for count in zero_counts]
    zero_counts_values.sort()
    n = len(zero_counts_values)
    max_value_index = int((n / 2) / (n + 1) * n)
    max_value = zero_counts_values[max_value_index]
    min_value = zero_counts_values[0]
    
    double_tap_blocks = []
    for start_block_id, end_block_id, zero_count in zero_counts:
        if min_value <= zero_count <= max_value:
            double_tap_blocks.append((start_block_id, end_block_id))
    return double_tap_blocks

def merge_continuous_double_taps(double_tap_blocks):
    """合并连续的双拍块对"""
    merged_double_tap_blocks = []
    current_start = None
    current_end = None
    
    for start_block_id, end_block_id in double_tap_blocks:
        if current_start is None:
            current_start = start_block_id
            current_end = end_block_id
        elif start_block_id == current_end:
            current_end = end_block_id
        else:
            merged_double_tap_blocks.append((current_start, current_end))
            current_start = start_block_id
            current_end = end_block_id
    
    if current_start is not None:
        merged_double_tap_blocks.append((current_start, current_end))
    
    return merged_double_tap_blocks

def merge_double_taps(df, block_info, double_tap_blocks, start_block_id):
    """合并确定的双拍数据块"""
    merged_double_tap_blocks = merge_continuous_double_taps(double_tap_blocks)
    merged_blocks = []
    new_block_id = start_block_id

    for start_block_id, end_block_id in merged_double_tap_blocks:
        start_row = next(bi[1] for bi in block_info if bi[0] == start_block_id)
        end_row = next(bi[2] for bi in block_info if bi[0] == end_block_id)
        
        merged_block = df.iloc[start_row:end_row + 1].copy()
        merged_block['block_id'] = new_block_id  # 使用新的连续编号
        merged_block['touch_type'] = 'DT'  # 标记为双拍
        
        merged_blocks.append(merged_block)
        new_block_id += 1

    return merged_blocks, new_block_id

def process_double_tap_folder(df, file_path, tactile_type, start_block_id):
    """处理双拍数据，将挨得非常近的两个连续块标记为双拍类型"""
    blocks, block_info = process_labeledDta(file_path, tactile_type)
    zero_counts = calculate_zero_counts(df, block_info)
    double_tap_blocks = determine_double_taps(zero_counts)
    merged_blocks, new_block_id = merge_double_taps(df, block_info, double_tap_blocks, start_block_id)
    return merged_blocks, new_block_id

def process_single_folder(data_folder, folder_name, start_block_id):
    """处理指定文件夹下的labeled_data.csv，统计数据块个数，并打印块的开始行和结束行"""
    folder_path = os.path.join(data_folder, folder_name)
    
    if os.path.isdir(folder_path):  # Check if it's a directory
        file_path = os.path.join(folder_path, 'labeled_data.csv')
        
        if os.path.isfile(file_path):
            tactile_type = folder_name.split('-')[1][1:]
            df = pd.read_csv(file_path)
            
            if tactile_type == 'DT':
                blocks, new_block_id = process_double_tap_folder(df, file_path, tactile_type, start_block_id)
            elif tactile_type in ['ST', 'G', 'P']:
                blocks, _ = process_labeledDta(file_path, tactile_type)
                new_block_id = start_block_id + len(blocks)
            else:
                blocks = []
                new_block_id = start_block_id
            
            return blocks, new_block_id
        else:
            print(f"文件夹 {folder_name} 下没有找到labeled_data.csv文件")
            return [], start_block_id
    else:
        print(f"{folder_name} 不是有效的文件夹")
        return [], start_block_id

def process_all_folders(data_folder, output_file):
    """处理所有子文件夹，合并所有数据块，生成tactile_dataset.csv文件"""
    all_blocks = []
    global global_block_id
    global_block_id = 0  # Reset global_block_id at the start
    
    for folder_name in os.listdir(data_folder):
        folder_blocks, global_block_id = process_single_folder(data_folder, folder_name, global_block_id)
        all_blocks.extend(folder_blocks)
    
    if all_blocks:
        result_df = pd.concat(all_blocks, ignore_index=True)
        output_path = os.path.join(output_file, 'tactile_dataset.csv')
        
        # 删除已存在的文件
        if os.path.exists(output_path):
            os.remove(output_path)
        
        # 保存新文件
        result_df.to_csv(output_path, index=False)
        print(f"tactile_dataset.csv 文件已生成，包含 {len(all_blocks)} 个数据块")
    else:
        print("没有数据块被处理")

# 设置数据文件夹路径
data_folder = '../DATA/Labeled_data'
output_file = '../DATA/'
process_all_folders(data_folder, output_file)

tactile_dataset.csv 文件已生成，包含 511 个数据块


In [20]:
tactile_df = pd.read_csv("/Users/shangyu/Documents/GitHub/tactileGestureDetection/DATA/tactile_dataset.csv")
tactile_df[tactile_df['block_id']==6]
# tactile_df_sorted = tactile_df.sort_values(by='time', ascending=True)
# tactile_df_sorted

# # 检查时间列是否有重复值
# duplicate_times = tactile_df['time'].duplicated()
# duplicate_times

# # # 统计重复值的数量
# # num_duplicates = duplicate_times.sum()

# # # 打印结果
# # print(f"Number of duplicate time values: {num_duplicates}")

# # 找出时间列中重复的值
# duplicate_times = tactile_df[tactile_df['time'].duplicated(keep=False)]

# # 显示这些重复的值及其对应的行
# duplicate_times


,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,etau_J0,etau_J1,etau_J2,etau_J3,etau_J4,etau_J5,etau_J6,label,block_id,touch_type
228,1796,17.949889,0.188269,-16.649282,-1.593821,20.656504,-0.048195,2.006725,-0.151195,0.0,...,-0.188269,16.649282,1.593821,-20.656504,0.048195,-2.006725,0.151195,1,6,DT
229,1797,17.959906,1.084876,-17.145193,-0.772593,22.414011,0.189842,2.687878,-0.089855,0.0,...,-1.084876,17.145193,0.772593,-22.414011,-0.189842,-2.687878,0.089855,1,6,DT
230,1798,17.969912,1.084876,-17.145193,-0.772593,22.414011,0.189842,2.687878,-0.089855,0.0,...,-1.084876,17.145193,0.772593,-22.414011,-0.189842,-2.687878,0.089855,1,6,DT
231,1799,17.979906,1.382422,-18.137014,-0.586131,22.263254,0.070823,2.708935,-0.008373,0.0,...,-1.382422,18.137014,0.586131,-22.263254,-0.070823,-2.708935,0.008373,0,6,DT
232,1800,17.989897,1.120581,-18.101309,-0.847972,21.402353,0.098289,2.666821,-0.049572,0.0,...,-1.120581,18.101309,0.847972,-21.402353,-0.098289,-2.666821,0.049572,0,6,DT
233,1801,17.999892,0.711951,-17.105520,-1.145518,20.390696,0.171531,2.593578,-0.079784,0.0,...,-0.711951,17.105520,1.145518,-20.390696,-0.171531,-2.593578,0.079784,0,6,DT
234,1802,18.009898,0.672278,-16.994436,-1.109813,17.998423,0.226463,2.426037,-0.099926,0.0,...,-0.672278,16.994436,1.109813,-17.998423,-0.226463,-2.426037,0.099926,0,6,DT
235,1803,18.019892,0.561194,-17.335623,-1.331981,16.161570,0.217307,2.258495,-0.099926,0.0,...,-0.561194,17.335623,1.331981,-16.161570,-0.217307,-2.258495,0.099926,0,6,DT
236,1804,18.029891,0.148596,-17.299917,-1.815989,16.574167,0.281394,2.341808,-0.038585,0.0,...,-0.148596,17.299917,1.815989,-16.574167,-0.281394,-2.341808,0.038585,0,6,DT
237,1805,18.039888,-0.339380,-17.069815,-2.192882,18.109507,0.336326,2.551464,-0.059643,0.0,...,0.339380,17.069815,2.192882,-18.109507,-0.336326,-2.551464,0.059643,0,6,DT


In [9]:
# 双拍数据分割测试代码
import os
import pandas as pd
import numpy as np

global_block_id = 0

def process_labeledDta(file_path, tactile_type):
    """处理单个labeled_data.csv文件，将连续label为1的数据块标记为指定触碰类型"""
    global global_block_id
    df = pd.read_csv(file_path)
    labeled_blocks = []

    in_block = False
    block_start = 0
    block_info = []

    for i in range(len(df)):
        if df.iloc[i, df.columns.get_loc('label')] == 1 and not in_block:
            in_block = True
            block_start = i
        elif df.iloc[i, df.columns.get_loc('label')] == 0 and in_block:
            in_block = False
            block = df.iloc[block_start:i].copy()
            block['touch_type'] = tactile_type
            block['block_id'] = global_block_id  # 添加全局唯一数据块标识符
            labeled_blocks.append(block)
            block_info.append((global_block_id, block_start, i - 1))  # 记录块的ID、开始行和结束行
            global_block_id += 1  # 增加全局唯一数据块标识符

    if in_block:
        block = df.iloc[block_start:].copy()
        block['touch_type'] = tactile_type
        block['block_id'] = global_block_id  # 添加全局唯一数据块标识符
        labeled_blocks.append(block)
        block_info.append((global_block_id, block_start, len(df) - 1))  # 记录块的ID、开始行和结束行
        global_block_id += 1  # 增加全局唯一数据块标识符
    
    return labeled_blocks, block_info

def calculate_zero_counts(df, block_info):
    """计算相邻数据块之间0的个数"""
    zero_counts = []
    for i in range(len(block_info) - 1):
        end_of_current_block = block_info[i][2]
        start_of_next_block = block_info[i + 1][1]
        zero_count = (df.iloc[end_of_current_block + 1:start_of_next_block]['label'] == 0).sum()
        zero_counts.append((block_info[i][0], block_info[i + 1][0], zero_count))
    return zero_counts

def determine_double_taps(zero_counts):
    """根据0的个数决定哪些单拍应该合并为双拍"""
    zero_counts_values = [count[2] for count in zero_counts]
    zero_counts_values.sort()
    print(f"排序后的 0值排序是 {zero_counts_values}")
    n = len(zero_counts_values)
    print(f"n 值是 {n}")
    max_value_index = int((n / 2) / (n + 1) * n)
    print(f"max_index is {max_value_index}")
    max_value = zero_counts_values[max_value_index]
    print(f"最大值是：{max_value}")
    min_value = zero_counts_values[0]
    print(f"最小值是：{min_value}")
    
    double_tap_blocks = []
    for start_block_id, end_block_id, zero_count in zero_counts:
        if min_value <= zero_count <= max_value:
            double_tap_blocks.append((start_block_id, end_block_id))
    return double_tap_blocks

def merge_continuous_double_taps(double_tap_blocks):
    """合并连续的双拍块对"""
    merged_double_tap_blocks = []
    current_start = None
    current_end = None
    
    for start_block_id, end_block_id in double_tap_blocks:
        if current_start is None:
            current_start = start_block_id
            current_end = end_block_id
        elif start_block_id == current_end:
            current_end = end_block_id
        else:
            merged_double_tap_blocks.append((current_start, current_end))
            current_start = start_block_id
            current_end = end_block_id
    
    if current_start is not None:
        merged_double_tap_blocks.append((current_start, current_end))
    
    return merged_double_tap_blocks

def merge_double_taps(df, block_info, double_tap_blocks):
    """合并确定的双拍数据块"""
    merged_double_tap_blocks = merge_continuous_double_taps(double_tap_blocks)
    merged_blocks = []
    new_block_id = block_info[0][0]  # 从第一个 block_id 开始连续编号
    
    for start_block_id, end_block_id in merged_double_tap_blocks:
        start_row = block_info[start_block_id][1]
        end_row = block_info[end_block_id][2]
        
        merged_block = df.iloc[start_row:end_row + 1].copy()
        merged_block['block_id'] = new_block_id  # 使用新的连续编号
        merged_block['touch_type'] = 'DT'  # 标记为双拍
        
        merged_blocks.append(merged_block)
        new_block_id += 1
    
    return merged_blocks

def process_double_tap_folder(data_folder, folder_name):
    """处理双拍数据，将挨得非常近的两个连续块标记为双拍类型"""
    folder_path = os.path.join(data_folder, folder_name)

    print(os.path.isdir(folder_path))
    
    if os.path.isdir(folder_path):  # Check if it's a directory
        
        file_path = os.path.join(folder_path, 'labeled_data.csv')
        
        if os.path.isfile(file_path):
            tactile_type = folder_name.split('-')[1][1:]
            df = pd.read_csv(file_path)
            blocks, block_info = process_labeledDta(file_path, tactile_type)
            print(f"block_info 是 {block_info}")
            zero_counts = calculate_zero_counts(df, block_info)
            print(f"所有数据块之间0的个数是 {zero_counts}")

            double_tap_blocks = determine_double_taps(zero_counts)
            print(f"属于一组的 block 是 {double_tap_blocks}")
            merged_blocks = merge_double_taps(df, block_info, double_tap_blocks)
            
            # 打印数据块的个数
            block_count = len(merged_blocks)
            print(f"文件夹 {folder_name} 中的数据块个数为: {block_count}")
            
            # 打印合并后的数据块的开始行和结束行
            for block in merged_blocks:
                start_row = block.index[0]
                end_row = block.index[-1]
                block_id = block['block_id'].values[0]
                print(f"数据块 {block_id} 从行 {start_row} 到行 {end_row}")

            # 将合并后的数据块保存为 CSV 文件
            output_df = pd.concat(merged_blocks, ignore_index=True)
            output_file = os.path.join(folder_path, f'{folder_name}_contactBlock.csv')
            output_df.to_csv(output_file, index=False)
            print(f"合并后的双拍数据已保存到: {output_file}")
            
        else:
            print(f"文件夹 {folder_name} 下没有找到labeled_data.csv文件")
    else:
        print(f"{folder_name} 不是有效的文件夹")


# 指定文件夹
data_folder = '../DATA/Labeled_data'
folder_name = '0724-7DT-S1'  # 替换为要处理的文件夹名称
process_double_tap_folder(data_folder, folder_name)

True
block_info 是 [(0, 401, 409), (1, 427, 437), (2, 754, 762), (3, 775, 784), (4, 1180, 1191), (5, 1201, 1212), (6, 2295, 2304), (7, 2318, 2327), (8, 2629, 2639), (9, 2655, 2669), (10, 2939, 2951), (11, 2965, 2976), (12, 3316, 3325), (13, 3343, 3354), (14, 3767, 3775), (15, 3793, 3793), (16, 4179, 4190), (17, 4203, 4218), (18, 4572, 4583), (19, 4597, 4602), (20, 4947, 4948), (21, 4954, 4955), (22, 4973, 4986), (23, 5313, 5326), (24, 5341, 5353), (25, 5675, 5688), (26, 5704, 5716), (27, 6125, 6133), (28, 6150, 6151), (29, 6524, 6535), (30, 6549, 6563), (31, 6867, 6876), (32, 6894, 6898), (33, 7213, 7223), (34, 7238, 7252), (35, 7542, 7550), (36, 7567, 7574), (37, 7891, 7895), (38, 7915, 7923), (39, 8213, 8222), (40, 8239, 8252), (41, 8638, 8650), (42, 8663, 8679), (43, 8981, 8987), (44, 9003, 9013), (45, 9345, 9347), (46, 9370, 9378), (47, 9671, 9683), (48, 9697, 9700), (49, 9996, 10004), (50, 10025, 10039), (51, 10371, 10380), (52, 10397, 10407), (53, 10762, 10763), (54, 10789, 10798)

In [16]:
#(ST,G,P)单个数据集的切分代码
import os
import pandas as pd

global_block_id = 0

def process_labeledDta(file_path, tactile_type):
    """处理单个labeled_data.csv文件，将连续label为1的数据块标记为指定触碰类型"""
    global global_block_id
    df = pd.read_csv(file_path)
    # print(f"现在读的数据来自 {file_path}")
    labeled_blocks = []

    in_block = False
    block_start = 0
    block_info = []
    zero_counts = []

    for i in range(len(df)):
        if df.iloc[i, df.columns.get_loc('label')] == 1 and not in_block:
            in_block = True
            block_start = i
        elif df.iloc[i, df.columns.get_loc('label')] == 0 and in_block:
            in_block = False
            block = df.iloc[block_start:i].copy()
            block['touch_type'] = tactile_type
            block['block_id'] = global_block_id  # 添加全局唯一数据块标识符
            labeled_blocks.append(block)
            block_info.append((global_block_id, block_start, i - 1))  # 记录块的ID、开始行和结束行
            global_block_id += 1  # 增加全局唯一数据块标识符

    if in_block:
        block = df.iloc[block_start:].copy()
        block['touch_type'] = tactile_type
        block['block_id'] = global_block_id  # 添加全局唯一数据块标识符
        labeled_blocks.append(block)
        block_info.append((global_block_id, block_start, len(df) - 1))  # 记录块的ID、开始行和结束行
        global_block_id += 1  # 增加全局唯一数据块标识符

        # 计算偶数数据块和相邻奇数数据块之间0的个数
    for i in range(0, len(block_info) - 1, 2):
        end_of_even_block = block_info[i][2]
        start_of_odd_block = block_info[i + 1][1]
        zero_count = (df.iloc[end_of_even_block + 1:start_of_odd_block]['label'] == 0).sum()
        zero_counts.append((block_info[i][0], block_info[i + 1][0], zero_count))
    
    return labeled_blocks, block_info, zero_counts

def process_single_folder(data_folder, folder_name):
    """处理指定文件夹下的labeled_data.csv，统计数据块个数，并打印块的开始行和结束行"""
    folder_path = os.path.join(data_folder, folder_name)
    
    if os.path.isdir(folder_path):  # Check if it's a directory
        file_path = os.path.join(folder_path, 'labeled_data.csv')
        
        if os.path.isfile(file_path):
            tactile_type = folder_name.split('-')[1][1:]
            
            if tactile_type in ['ST','G','P']:
                blocks, block_info,zero_counts = process_labeledDta(file_path, tactile_type)
                
                # 打印数据块的个数
                block_count = len(blocks)
                print(f"文件夹 {folder_name} 中的数据块个数为: {block_count}")


                # 将所有数据块合并为一个DataFrame
                combined_df = pd.concat(blocks)
                
                # 保存到CSV文件
                output_file = os.path.join(folder_path, f'{folder_name}_contactBlock.csv')
                combined_df.to_csv(output_file, index=False)
                print(f"文件已保存至: {output_file}")
                
                # 打印数据块的开始行和结束行
                # for block_id, start_row, end_row in block_info:
                    # print(f"数据块 {block_id} 从行 {start_row} 到行 {end_row}")

                # 打印偶数数据块和相邻奇数数据块之间0的个数
                # for even_block_id, odd_block_id, zero_count in zero_counts:
                #     print(f"数据块 {even_block_id} 和数据块 {odd_block_id} 之间有 {zero_count} 个0")
        else:
            print(f"文件夹 {folder_name} 下没有找到labeled_data.csv文件")
    else:
        print(f"{folder_name} 不是有效的文件夹")

# 指定文件夹
data_folder = '../DATA/Labeled_data'
folder_name = '0724-7ST-Y1'  # 替换为要处理的文件夹名称
process_single_folder(data_folder, folder_name)

文件夹 0724-7ST-Y1 中的数据块个数为: 62
文件已保存至: ../DATA/Labeled_data/0724-7ST-Y1/0724-7ST-Y1_contactBlock.csv


In [ ]:
# 数据块的最大时差
import pandas as pd
data_path = 'DATA/tactile_dataset.csv'
data = pd.read_csv(data_path)
block_ids = data['block_id'].unique()
# block_ids = data['block_id'].nunique()
# block_ids
duritions = {}
for id in block_ids:
    # print(f" id is {id}")
    data_ = data[data['block_id']==id]
    first_time = data_.iloc[0]['time']
    # print(f"first_time is {first_time}")
    last_time = data_.iloc[-1]['time']
    # print(f"last_time is {last_time}")
    durition = last_time - first_time
# print(duration)
    if id in duritions.keys():
        duritions[id] = durition
    else:
        duritions[id] = durition
print(duritions)

max_durition = max(duritions.values())
max_durition_block = [key for key,value in duritions.items() if value == max_durition ]
print(f"the max_durition is {max_durition} and the block_id is {max_durition_block}")